<a href="https://colab.research.google.com/github/Data-Science-and-Data-Analytics-Courses/UCSanDiegoX---Machine-Learning-Fundamentals-03-Jan-2019-audit/blob/master/Packages/Files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from contextlib import contextmanager
from functools import partial
import requests
from urllib.parse import urlsplit
from pathlib import Path
from fnmatch import fnmatch, filter as fnfilter
import io
import tarfile, gzip, zipfile
import numpy as np
from IPython.display import display

# General

In [0]:
def download(url, dest=".", name="", force=False, options="-#"):
  """
  Download url into dest
  name: if provided, rename downloaded
  force: if True, overwrite existing resource
  options: supported by curl (https://curl.haxx.se/docs/manpage.html)
  """
  
  rurl = urlsplit(url)
  dest = Path(dest).resolve()
  downloaded = dest / (name or Path(rurl.path).name)
  
  # Download
  if not downloaded.exists() or force:
    !curl $options --create-dirs "{rurl.geturl()}" -o "{downloaded}"

  return downloaded

if __name__ == "__main__":
  url="http://yann.lecun.com/exdb/mnist/train-images-idx3-ubyte.gz"
  download(url)

/content/train-images-idx3-ubyte.gz already exists


# Tar

In [0]:
@contextmanager
def readtar(fpath, pattern="*.*", mode="r", peek=False):
  """
  Read pattern in fpath
  mode: supported by tarfile.open (https://docs.python.org/3/library/tarfile.html#tarfile.open)
  peek: if True, show matches before reading
  """

  t = tarfile.open(fpath, mode=mode)
  try:
    matches = fnfilter(t.getnames(), pattern)
    if peek:
      display("Read", matches)
      input("Press Enter to continue...")
    yield map(t.extractfile, matches)
  finally:
    t.close()

In [0]:
def runtar(url, dest=".", patterns=None, mode="r", top=True, force=False, peek=False):
  """
  Retrieve url and untar patterns into dest
  mode: supported by tarfile.open (https://docs.python.org/3/library/tarfile.html#tarfile.open)
  top: if True, create top directory if not exist
  force: if True, overwrite existing members
  peek: if True, show members before extraction
  """

  rurl = urlsplit(url)
  dest = Path(dest).resolve()

  # Retrieve
  r = requests.get(url)
  t = tarfile.open(fileobj=io.BytesIO(r.content), mode=mode)

  # Members
  members = np.array(t.getnames())
  items = np.vectorize(Path)(members) # Path objects
  extract = members

  # Top directory
  tops = map(lambda m: m.parts[0], items) # tops of directory tree
  if top and len(set(tops))>=2:
    dest = dest / Path(rurl.path).stem # filename without .zip

  # Matches
  if patterns:
    matched = list(map(lambda m: any(map(m.match, patterns)), items))
    extract = extract[matched]
  
  # Existing members
  extracted = list(map(dest.joinpath, extract))
  if not force:
    exist = list(map(Path.exists, extracted))
    extract = extract[np.logical_not(exist)]

  # Show members
  if peek:
    display("Extract", extract, f"into {dest}")
    input("Press Enter to continue... ")
  
  # Extract
  list(map(partial(z.extract, path=dest), extract))
  
  return extracted # extracted/existing
  
  
  # Members
  members = t.getnames()
  if peek:
    display("Extract", members)
    dest = Path(input(f"into {dest} (or specify): ") or dest)
  
  # Extract
  extracted = list(map(dest.joinpath, members))
  t.extractall(dest)

  return extracted

if __name__ == "__main__":
  url = "https://cvml.ist.ac.at/AwA/AwA-base.tar.bz2"
  runtar(url, mode="r:bz2", peek=True)

'Extract'

['Animals_with_Attributes/README-attributes.txt',
 'Animals_with_Attributes/Features/README-features.txt',
 'Animals_with_Attributes/classes.txt',
 'Animals_with_Attributes/predicates.txt',
 'Animals_with_Attributes/testclasses.txt',
 'Animals_with_Attributes/trainclasses.txt',
 'Animals_with_Attributes/predicate-matrix-binary.txt',
 'Animals_with_Attributes/predicate-matrix-continuous.txt',
 'Animals_with_Attributes/predicate-matrix.png',
 'Animals_with_Attributes/lampert-cvpr2009.pdf']

into /content (or specify): 


# Zip

In [22]:
def runzip(url, dest=".", patterns=None, top=True, force=False, peek=False):
  """
  Retrieve url and unzip patterns into dest
  top: if True, create top directory if not exist
  force: if True, overwrite existing members
  peek: if True, show members before extraction
  """
  
  rurl = urlsplit(url)
  dest = Path(dest).resolve()

  # Retrieve
  r = requests.get(url)
  z = zipfile.ZipFile(io.BytesIO(r.content))

  # Members
  members = np.array(z.namelist())
  items = np.vectorize(Path)(members) # Path objects
  extract = members

  # Top directory
  tops = map(lambda m: m.parts[0], items) # tops of directory tree
  if top and len(set(tops))>=2:
    dest = dest / Path(rurl.path).stem # filename without .zip

  # Matches
  if patterns:
    matched = list(map(lambda m: any(map(m.match, patterns)), items))
    extract = extract[matched]
  
  # Existing members
  extracted = list(map(dest.joinpath, extract))
  if not force:
    exist = list(map(Path.exists, extracted))
    extract = extract[np.logical_not(exist)]

  # Show members
  if peek:
    display("Extract", extract, f"into {dest}")
    input("Press Enter to continue... ")
  
  # Extract
  list(map(partial(z.extract, path=dest), extract))
  
  return extracted # extracted/existing

if __name__ == "__main__":
  url = "https://prod-edxapp.edx-cdn.org/assets/courseware/v1/e7b080a506546a859b50d03f9d0705f5/asset-v1:UCSanDiegoX+DSE220x+1T2019+type@asset+block/DSE220x_PA7.zip"
  runzip(url, peek=True)

'Extract'

array([], dtype='<U84')

'into /content/DSE220x_PA7'

Press Enter to continue... 
